In [1]:
import featuretools as ft
import pandas as pd

In [2]:
# Create dataframe for Customers entity
Customers = pd.DataFrame({
    'CustomerID': [101, 102, 103],
    'Name': ['John Doe', 'Jane Smith', 'Mike Jordan'],
    'Email': ['john.doe@example.com', 'jane.smith@example.com', 'mike.jordan@example.com'],
    'SignupDate': ['2023-01-10', '2023-01-15', '2023-01-20']
})

# Create dataframe for Products entity
Products = pd.DataFrame({
    'ProductID': [201, 202, 203],
    'Name': ['Laptop', 'Tablet', 'Smartphone'],
    'Category': ['Electronics', 'Electronics', 'Electronics'],
    'Price': [1000, 500, 800]
})

# Create dataframe for Orders entity
Orders = pd.DataFrame({
    'OrderID': [301, 302, 303],
    'CustomerID': [101, 102, 103],
    'OrderDate': ['2023-02-01', '2023-02-05', '2023-02-10'],
    'ShipDate': ['2023-02-03', '2023-02-07', '2023-02-12']
})

# Create dataframe for OrderDetails entity
OrderDetails = pd.DataFrame({
    'OrderID': [301, 302, 303],
    'ProductID': [201, 202, 203],
    'Quantity': [1, 2, 1],
    'Discount': [0, 0.1, 0]
})

In [3]:
# Define entity set 
es = ft.EntitySet(id='ecommerce_data')

# Add dataframe into the EntitySet
es = es.add_dataframe(dataframe_name= "Customers", dataframe= Customers, index="CustomerID", time_index="SignupDate")
es = es.add_dataframe(dataframe_name= "Products", dataframe= Products, index="ProductID")
es = es.add_dataframe(dataframe_name= "Orders", dataframe= Orders, index="OrderID", time_index="OrderDate")
es = es.add_dataframe(dataframe_name= "OrderDetails", dataframe= OrderDetails, index="OrderID, ProductID")

C:\Users\annjo\AppData\Roaming\Python\Python310\site-packages\woodwork\type_sys\utils.py:40: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  pd.to_datetime(
C:\Users\annjo\AppData\Roaming\Python\Python310\site-packages\woodwork\type_sys\utils.py:40: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  pd.to_datetime(
C:\Users\annjo\AppData\Roaming\Python\Python310\site-packages\woodwork\type_sys\utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, 

In [4]:
# Establish relationship between entities 
relationship_customer_order = es.add_relationship(parent_dataframe_name='Customers', parent_column_name='CustomerID', child_dataframe_name='Orders', child_column_name='CustomerID')
relationship_product_orderdetails = es.add_relationship(parent_dataframe_name='Products', parent_column_name='ProductID', child_dataframe_name='OrderDetails', child_column_name='ProductID')
relationship_order_orderdetails = es.add_relationship(parent_dataframe_name='Orders', parent_column_name='OrderID', child_dataframe_name='OrderDetails', child_column_name='OrderID')
print(es)

Entityset: ecommerce_data
  DataFrames:
    Customers [Rows: 3, Columns: 4]
    Products [Rows: 3, Columns: 4]
    Orders [Rows: 3, Columns: 4]
    OrderDetails [Rows: 3, Columns: 5]
  Relationships:
    Orders.CustomerID -> Customers.CustomerID
    OrderDetails.ProductID -> Products.ProductID
    OrderDetails.OrderID -> Orders.OrderID


In [5]:
# Perform Deep Feature Synthesis
feature_matrix, feature_defs = ft.dfs(entityset=es, target_dataframe_name='Orders')
feature_matrix

,CustomerID,COUNT(OrderDetails),MAX(OrderDetails.Discount),MAX(OrderDetails.Quantity),MEAN(OrderDetails.Discount),MEAN(OrderDetails.Quantity),MIN(OrderDetails.Discount),MIN(OrderDetails.Quantity),SKEW(OrderDetails.Discount),SKEW(OrderDetails.Quantity),...,Customers.SKEW(OrderDetails.Discount),Customers.SKEW(OrderDetails.Quantity),Customers.STD(OrderDetails.Discount),Customers.STD(OrderDetails.Quantity),Customers.SUM(OrderDetails.Discount),Customers.SUM(OrderDetails.Quantity),Customers.DAY(SignupDate),Customers.MONTH(SignupDate),Customers.WEEKDAY(SignupDate),Customers.YEAR(SignupDate)
OrderID,,,,,,,,,,,,,,,,,,,,,
301,101,1,0.0,1.0,0.0,1.0,0.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,1.0,10,1,1,2023
302,102,1,0.1,2.0,0.1,2.0,0.1,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.1,2.0,15,1,6,2023
303,103,1,0.0,1.0,0.0,1.0,0.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,1.0,20,1,4,2023


In [7]:
feature_defs

[<Feature: CustomerID>,
 <Feature: COUNT(OrderDetails)>,
 <Feature: MAX(OrderDetails.Discount)>,
 <Feature: MAX(OrderDetails.Quantity)>,
 <Feature: MEAN(OrderDetails.Discount)>,
 <Feature: MEAN(OrderDetails.Quantity)>,
 <Feature: MIN(OrderDetails.Discount)>,
 <Feature: MIN(OrderDetails.Quantity)>,
 <Feature: SKEW(OrderDetails.Discount)>,
 <Feature: SKEW(OrderDetails.Quantity)>,
 <Feature: STD(OrderDetails.Discount)>,
 <Feature: STD(OrderDetails.Quantity)>,
 <Feature: SUM(OrderDetails.Discount)>,
 <Feature: SUM(OrderDetails.Quantity)>,
 <Feature: DAY(OrderDate)>,
 <Feature: DAY(ShipDate)>,
 <Feature: MONTH(OrderDate)>,
 <Feature: MONTH(ShipDate)>,
 <Feature: WEEKDAY(OrderDate)>,
 <Feature: WEEKDAY(ShipDate)>,
 <Feature: YEAR(OrderDate)>,
 <Feature: YEAR(ShipDate)>,
 <Feature: MAX(OrderDetails.Products.Price)>,
 <Feature: MEAN(OrderDetails.Products.Price)>,
 <Feature: MIN(OrderDetails.Products.Price)>,
 <Feature: SKEW(OrderDetails.Products.Price)>,
 <Feature: STD(OrderDetails.Products.Pr